In [ ]:
!pip install tensorly
!pip install opencv-python

In [ ]:
!pip install --force-reinstall -v "tensorly==0.7.0"

In [ ]:
!pip install --force-reinstall -v "numpy==1.23.5"

In [ ]:
import logging
from typing import Type

import numpy as np
import warnings
import os
import cv2
import pickle
import pandas as pd
import scipy as sp
import scipy.misc
from matplotlib import pyplot as plt
from glob import glob
import tensorly as tl
import numpy as np
from sklearn.linear_model import orthogonal_mp
from tensorly.decomposition import tucker, partial_tucker
from sklearn.feature_extraction.image import extract_patches_2d, reconstruct_from_patches_2d
warnings.filterwarnings('ignore')
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

logging.basicConfig(level=logging.INFO)

In [ ]:
class Pursuit:
    """
    Algorithms that inherit from this class are methods to solve problems of the like
    \min_A \| DA - Y \|_2 s.t. \|A\|_0 <= t.
    Here, D is a given dictionary of size (n x K)
    Y is a given matrix of size (n x N), where N is the number of samples
    The Pursuit will return a matrix A of size (K x N).
    """

    def __init__(self, dictionary, max_iter=False, tol=None, sparsity=None):
        self.D = np.array(dictionary.matrix)
        self.max_iter = max_iter
        self.tol = tol
        self.sparsity = sparsity
        if (self.tol is None and self.sparsity is None) or (self.tol is not None and self.sparsity is not None):
            raise ValueError("blub")
        self.data = None
        self.alphas = []

    def fit(self, Y):
        return [], self.alphas

In [ ]:
class OrthogonalMatchingPursuit(Pursuit):
    """
    Wrapper for orthogonal_mp from scikit-learn
    """

    def fit(self, Y):
        return orthogonal_mp(self.D, Y, n_nonzero_coefs=self.sparsity,
                             tol=self.tol, precompute=True)

In [ ]:
class Dictionary:
    """
    The Dictionary class is more or less a wrapper around the numpy array class. It holds a numpy ndarray in
    the attribute `matrix` and adds some useful functions for it. The dictionary elements can be accessed
    either by D.matrix[i,j] or directly through D[i,j].
    """

    def __init__(self, matrix):
        self.matrix = np.array(matrix)
        self.shape = matrix.shape

    def __getitem__(self, item):
        return self.matrix[item]

    def is_unitary(self):
        """
        Checks whether the dictionary is unitary.
        Returns:
            True, if the dicitonary is unitary.
        """
        n, K = self.shape
        if n == K:
            return np.allclose(np.dot(self.matrix.T, self.matrix), np.eye(n))
        else:
            return False

    def is_normalized(self):
        """
        Checks wheter the dictionary is l2-normalized.
        Returns:
            True, if dictionary is l2-normalized.
        """
        n, K = self.shape
        return np.allclose([np.linalg.norm(self.matrix[:, i]) for i in range(K)], np.ones(K))


    def mutual_coherence(self):
        """
        Computes the dictionary's mutual coherence.
        Returns:
            Mutual coherence
        """
        return np.max(self._mutual_coherence(self.matrix))

    @staticmethod
    def _mutual_coherence(D):
        n, K = D.shape
        mu = [np.abs(np.dot(D[:, i].T, D[:, j]) /
                     (np.linalg.norm(D[:, i]) * np.linalg.norm(D[:, j])))
              for i in range(K) for j in range(K) if j != i]
        return mu

    def to_img(self):
        """
        Transforms the dictionary columns into patches and orders them for plotting purposes.
        Returns:
            Reordered dictionary matrix
        """
        # dictionary dimensions
        D = self.matrix
        n, K = D.shape
        M = self.matrix
        # stretch atoms
        for k in range(K):
            M[:, k] = M[:, k] - (M[:, k].min())
            if M[:, k].max():
                M[:, k] = M[:, k] / D[:, k].max()

        # patch size
        n_r = int(np.sqrt(n))

        # patches per row / column
        K_r = int(np.sqrt(K))

        # we need n_r*K_r+K_r+1 pixels in each direction
        dim = n_r * K_r + K_r + 1
        V = np.ones((dim, dim)) * np.min(D)

        # compute the patches
        patches = [np.reshape(D[:, i], (n_r, n_r)) for i in range(K)]

        # place patches
        for i in range(K_r):
            for j in range(K_r):
                V[j * n_r + 1 + j:(j + 1) * n_r + 1 + j, i * n_r + 1 + i:(i + 1) * n_r + 1 + i] = patches[
                    i * K_r + j]
        return V

In [ ]:
class KSVD:
    """
    Implements the original K-SVD Algorithm as described in [1].
    [1] Aharon, M., Elad, M. and Bruckstein, A., 2006. K-SVD: An algorithm for designing overcomplete dictionaries for
        sparse representation. IEEE Transactions on signal processing, 54(11), p.4311.
    Args:
        dictionary: Initial dictionary of type sparselandtools.dictionaries.Dictionary
        pursuit: Pursuit method to be used (any method from sparselandtools.pursuits)
        sparsity: Target sparsity
        noise_gain: Target noise_gain. If set, this will override the target sparsity
        sigma: Signal or image noise standard deviation.
    """

    def __init__(self, dictionary: Dictionary, pursuit: Type[Pursuit], sparsity: int, noise_gain=None, sigma=None):
        self.dictionary = Dictionary(dictionary.matrix)
        self.alphas = None
        self.pursuit = pursuit
        self.sparsity = sparsity
        self.noise_gain = noise_gain
        self.sigma = sigma
        self.original_image = None
        self.sparsity_values = []
        self.mses = []
        self.ssims = []
        self.psnrs = []
        self.iter = None

    def sparse_coding(self, Y: np.ndarray):
        logging.info("Entering sparse coding stage...")
        if self.noise_gain and self.sigma:
            p = self.pursuit(self.dictionary, tol=(self.noise_gain * self.sigma))
        else:
            p = self.pursuit(self.dictionary, sparsity=self.sparsity)
        self.alphas = p.fit(Y)
        logging.info("Sparse coding stage ended.")

    def dictionary_update(self, Y: np.ndarray):
        # iterate rows
        D = self.dictionary.matrix
        n, K = D.shape
        R = Y - D.dot(self.alphas)
        for k in range(K):
            logging.info("Updating column %s" % k)
            wk = np.nonzero(self.alphas[k, :])[0]
            if len(wk) == 0:
                continue
            Ri = R[:,wk] + D[:,k,None].dot(self.alphas[None,k,wk])
            U, s, Vh = np.linalg.svd(Ri)
            D[:, k] = U[:, 0]
            self.alphas[k, wk] = s[0] * Vh[0, :]
            R[:, wk] = Ri - D[:,k,None].dot(self.alphas[None,k,wk])
        self.dictionary = Dictionary(D)

    def fit(self, Y: np.ndarray, iter: int):
        for i in range(iter):
            logging.info("Start iteration %s" % (i + 1))
            self.sparse_coding(Y)
            self.dictionary_update(Y)
        return self.dictionary, self.alphas

In [ ]:
def preprocess(image):
    core, factors = partial_tucker(tl.tensor(image), rank=image.shape, modes=[0, 1, 2])
    return factors[0]


def get_matrix(y, image_path, patch_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    component = preprocess(image)
    
    # Extract all reference patches from the image
    patches = extract_patches_2d(component, (patch_size, patch_size))
    data = patches.reshape(patches.shape[0], -1)
    y = np.vstack([y, data])
    
    return y
  
image_directory = 'images/images_squares_25'
image_paths = glob(os.path.join(image_directory, '*.png'))
from random import shuffle
shuffle(image_paths)

In [ ]:
image_paths = image_paths[:50]
patch_size = 4

y = np.zeros(patch_size * patch_size)
for image_path in image_paths:
    y = get_matrix(y, image_path, patch_size)
y = np.delete(y, 0, axis=0)

In [ ]:
test = y.T
u, s, v = np.linalg.svd(test)

In [ ]:
initial_dictionary = Dictionary(u)

In [ ]:
initial_dictionary.is_unitary()

In [ ]:
ksvd = KSVD(initial_dictionary, OrthogonalMatchingPursuit, 16)

In [ ]:
# второй аргумент - число итераций
learn_dict, coeff = ksvd.fit(test, 2)

In [ ]:
v = learn_dict.matrix

In [ ]:
plt.imshow(learn_dict.to_img(), cmap='gray')

In [ ]:
# вычисление фич
def get_features(x):
    f_mic = []
    f_mac = []
    for i in range(x.shape[0]):
        values = x[i]
        values = np.abs(values[values!=0])
        sigma, _, mean = sp.stats.lognorm.fit(values, loc=0)
        f_mic.append(np.exp(mean + 0.5*sigma**2))
        f_mac.append(values.shape[0])
    return f_mic + f_mac

In [ ]:
# сохраняем словарь на диск
np.save(open('dictionary1.npy', 'wb'), v)

In [ ]:
v = np.load(open('dictionary1.npy', 'rb'))

In [ ]:
# создание таблицы для записи фич
df = pd.read_csv('images/response.csv')
images = df['image'].tolist()
labels = df['label'].tolist()
image_paths = ['images/images_squares_25/'+image for image in images]
del df
image_paths = image_paths[:500]
columns = ['image']
for i in range(v.shape[1]):
    columns.append('f%d_mic'%(i+1))
for i in range(v.shape[1]):
    columns.append('f%d_mac'%(i+1))
columns.append('label')
data = {column: [] for column in columns}
df = pd.DataFrame(data)

In [ ]:
for i, image_path in enumerate(image_paths):
    y = np.zeros(patch_size * patch_size)
    y = get_matrix(y, image_path, patch_size)
    y = y.T
    print(i)
    X = orthogonal_mp(v, y, n_nonzero_coefs=8)
    print(i)
    features = get_features(X)
    data = [images[i]] + features + [labels[i]]
    data = {column: [datum] for column, datum in zip(columns, data)}
    tmp = pd.DataFrame(data)
    df = pd.concat([df, tmp], axis=0)
    print('%d/%d %s' % (i, len(image_paths), image_path))

df.to_csv('dataset2_1.csv', index=False)